In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
import numpy as np
import os
from PIL import Image

In [2]:
def load_image(image_path):
    img = Image.open(image_path)
    img = img.resize((224, 224))  # Resize image to a fixed size
    img = np.array(img) / 255.0   # Normalize the image
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return 

image_path = "path/to/your/image.jpg"
image = load_image(image_path)
print(image.shape) 